In [6]:
import numpy as np
import pandas as pd

# 1. Recommendation sytem

We load the csv data into a pandas dataframe and do a quick check of the data.

In [53]:
data = 'archive/vodclickstream_uk_movies_03.csv'
df = pd.read_csv(data, sep=',', header=0, index_col=0)
df.head()

,datetime,duration,title,genres,release_date,movie_id,user_id
58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


In [54]:
df.shape

(671736, 7)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 671736 entries, 58773 to 730508
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   datetime      671736 non-null  object 
 1   duration      671736 non-null  float64
 2   title         671736 non-null  object 
 3   genres        671736 non-null  object 
 4   release_date  671736 non-null  object 
 5   movie_id      671736 non-null  object 
 6   user_id       671736 non-null  object 
dtypes: float64(1), object(6)
memory usage: 41.0+ MB


We can see that there is data available for each user for the movies the user clicked on. We are interested in gathering the title and genre of the maximum top 10 movies that each user clicked on regarding the number of clicks. 

What are the top 10 most clicked on movies?

In [56]:
df.title.value_counts()[:10]

title
Black Mirror: Bandersnatch                      6489
Bright                                          3392
Avengers: Age of Ultron                         2898
Annihilation                                    2873
Bird Box                                        2676
Hot Fuzz                                        2674
Deadpool                                        2576
FYRE: The Greatest Party That Never Happened    2389
The Big Short                                   2204
The Hitman's Bodyguard                          2179
Name: count, dtype: int64

What are the top 10 most clicked on movies for each user?

In [40]:
clicks = df.groupby(['user_id'])[["title", "genres"]].value_counts().to_frame()
clicks

count
user_id    title              genres                                   
00004e2862 Hannibal           Crime, Drama, Thriller                  1
000052a0a0 Looper             Action, Drama, Sci-Fi, Thriller         9
           Frailty            Crime, Drama, Thriller                  3
           Jumanji            Adventure, Comedy, Family, Fantasy      3
           Resident Evil      Action, Horror, Sci-Fi                  2
...                                                                 ...
fffeac83be Fight Club         Drama                                   1
           Enemy at the Gates Drama, History, War                     1
ffff2c5f9e Hot Fuzz           Action, Comedy, Mystery, Thriller       1
           Forks Over Knives  Documentary                             1
ffffd36adf Dead Man Walking   Crime, Drama                            1

[502343 rows x 1 columns]

In [37]:
clicks.loc["7cdfd0e14a"].head(10)

,,count
title,genres,
Twilight,"Drama, Fantasy, Romance",88
The Twilight Saga: New Moon,"Adventure, Drama, Fantasy, Romance",32
The Twilight Saga: Eclipse,"Adventure, Drama, Fantasy, Romance",10
The Twilight Saga: Breaking Dawn: Part 1,"Adventure, Drama, Fantasy, Romance, Thriller",9
The Twilight Saga: Breaking Dawn: Part 2,"Adventure, Drama, Fantasy, Romance",6
Monster House,"Animation, Comedy, Family, Fantasy, Mystery",5
Someone Great,"Comedy, Romance",4
Suicide Squad,"Action, Adventure, Fantasy, Sci-Fi",4
The Hangover Part II,Comedy,3


We also investigate how many unique genres are there in the dataset and what are their values.

In [59]:
unique_genres = set()
for genres in df.genres:
    unique_genres.update(genres.split(", "))
print(unique_genres)

{'Sci-Fi', 'History', 'Action', 'Biography', 'Romance', 'Sport', 'NOT AVAILABLE', 'Documentary', 'Talk-Show', 'Horror', 'Comedy', 'Mystery', 'Music', 'Film-Noir', 'Fantasy', 'Musical', 'Reality-TV', 'Crime', 'Short', 'Family', 'Drama', 'Animation', 'News', 'Thriller', 'Adventure', 'Western', 'War'}


We want to remove the movies that have no genre information.

In [60]:
df = df[df.genres != 'NOT AVAILABLE']

## 1.2 Minhash Signatures

Using the movie genre and user_ids, we will implement our min-hash signatures so that users with similar interests in a genre appear in the same bucket.

In [61]:
# Get unique genres for each movie watched by a user
df.genres = df.genres.str.split(', ').apply(set)
# Get unique genres for each user
user_genres = df.groupby('user_id').genres.agg(lambda x: set.union(*x))

In [62]:
user_genres.head()

user_id
00004e2862                             {Thriller, Drama, Crime}
000052a0a0    {Horror, Comedy, Mystery, Music, Fantasy, Crim...
000090e7c8                          {Sci-Fi, Thriller, Mystery}
000118a755                                             {Horror}
000296842d                   {Mystery, Thriller, Sci-Fi, Drama}
Name: genres, dtype: object

We define two functions:
- `MinHashSignature` which takes in a set of genres for a user and returns the min-hash signature for that list of genres.
- `Similarity` which takes in two min-hash signatures and returns the similarity between the two signatures.

In [63]:
# Function to compute the Minhash signature for each user
def Minhash_Signature(user_genres, NUM_HASHES):
    all_genres = set.union(*user_genres)
    num_genres = len(all_genres)

    # Assign a unique index to each genre
    genre_to_int = {genre: idx for idx, genre in enumerate(all_genres)}

    # Generate random hash functions
    hash_funcs = [lambda x, a=a, b=b: (a * genre_to_int[x] + b) % num_genres 
                    for a, b in zip(np.random.randint(1, 100, NUM_HASHES),
                                    np.random.randint(1, 100, NUM_HASHES))]

    # Initialize signature with maximum value for each hash function
    user_signature = np.full((NUM_HASHES, user_genres.size), np.inf)

    # Iterate through each genre in the set
    # and update the signature with minimum hashed value for each genre
    for i, hash_func in enumerate(hash_funcs):
        for j, genres in enumerate(user_genres):
            for genre in genres:
                # Hash the genre indices using each hash function and update the signature
                hash_value = hash_func(genre)
                user_signature[i, j] = min(user_signature[i, j], hash_value)

    # Convert the signature to a pandas Series
    user_signature = pd.Series(index=user_genres.index, data=user_signature.T.tolist())
    return user_signature

def minhash_signature(user_genres, num_hashes):
    all_genres = set.union(*user_genres)
    num_genres = len(all_genres)
    genre_to_int = {genre: idx for idx, genre in enumerate(all_genres)}

    hash_funcs = [
        (np.random.randint(1, 100), np.random.randint(1, 100)) for _ in range(num_hashes)
    ]

    user_signature = np.full((len(user_genres), num_hashes), np.inf)

    for i, (a, b) in enumerate(hash_funcs):
        for j, genres in enumerate(user_genres):
            for genre in genres:
                hash_value = (a * genre_to_int[genre] + b) % num_genres
                user_signature[j, i] = min(user_signature[j, i], hash_value)

    return pd.Series(user_signature.tolist(), index=user_genres.index)


# Function to compute the similarity between two users
def Similarity(user1_signature, user2_signature):
    # Compute the fraction of matching values
    sim = (np.array(user1_signature) == np.array(user2_signature))
    return sim.sum() / sim.size

In [92]:
# Select the number of hash functions
NUM_HASHES = 15

# Compute the Minhash signature for each user
#signatures = Minhash_Signature(user_genres, NUM_HASHES)
signatures = minhash_signature(user_genres, NUM_HASHES)

signatures.head()

user_id
00004e2862    [5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 8.0, 2.0, 16.0,...
000052a0a0    [3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
000090e7c8    [17.0, 6.0, 0.0, 4.0, 0.0, 9.0, 8.0, 5.0, 6.0,...
000118a755    [15.0, 8.0, 20.0, 2.0, 22.0, 19.0, 18.0, 17.0,...
000296842d    [17.0, 6.0, 0.0, 4.0, 0.0, 0.0, 8.0, 2.0, 6.0,...
dtype: object

Now we can use the min-hash signatures to find similar users. We will show an example with two users and see if they are similar.

In [93]:
id1 = '00004e2862'
id2 = '000296842d'
user1_signature = signatures[id1]
user2_signature = signatures[id2]

Similarity(user1_signature, user2_signature)

0.5333333333333333

The similarity between the two users is 0.5, which indicates they do have similar interests.
Let's verify this by looking at the genres of the movies they clicked on.

In [94]:
print(df.genres.loc[df.user_id == id1])
print(df.genres.loc[df.user_id == id2])

282575    Crime, Drama, Thriller
Name: genres, dtype: object
578906    Drama, Mystery, Sci-Fi, Thriller
578960    Drama, Mystery, Sci-Fi, Thriller
579156    Drama, Mystery, Sci-Fi, Thriller
579199    Drama, Mystery, Sci-Fi, Thriller
579565    Drama, Mystery, Sci-Fi, Thriller
579712    Drama, Mystery, Sci-Fi, Thriller
579747    Drama, Mystery, Sci-Fi, Thriller
580135    Drama, Mystery, Sci-Fi, Thriller
Name: genres, dtype: object


Indeed, we can see that both the users enjoy the Drama and Thriller genres.

## 1.3 Locality-Sensitive Hashing (LSH)

Next, given some `user_ids`, we want to recommend at most five movies to the user to watch based on the movies clicked by similar users.

To recommend at most five movies given a `user_id`, we will use the following procedure:

- Identify the two most similar users to this user.
- If these two users have any movies in common, recommend those movies based on the total number of clicks by these users.
- If there are no more common movies, propose the most clicked movies by the most similar user first, followed by the other user.

In [73]:
user_movie_clicks = df.groupby(['user_id', 'title']).size()
user_movie_clicks.head()

user_id     title      
00004e2862  Hannibal       1
000052a0a0  Ant-Man        1
            Drive Angry    1
            Frailty        3
            Green Room     1
dtype: int64

In [90]:
# Function to compute the similarity between two users
def Recomend(target_user, user_signature, user_movie_clicks):
    # Drop the target user from the user_signature Series and compute each similarity
    similarities = user_signature.drop(target_user).apply(lambda x: Similarity(x, user_signature[target_user]))

    # Select the top 2 most similar users
    top2_similar_users = pd.Series(similarities).nlargest(2)

    # Get the movie clicks for the top 2 similar users
    similar_user_movie_clicks = [user_movie_clicks[top2_similar_users.index[i]] for i in range(2)]
    # Sort the movie clicks in descending order
    similar_user_movie_clicks = [similar_user_movie_clicks[i].sort_values(ascending=False) for i in range(2)]
    
    # Get the common movies watched by the top 2 similar users
    common_movies = set.intersection(*[set(similar_user_movie_clicks[i].index) for i in range(2)])
    common_movies = list(common_movies)

    # Get the total number of clicks for the common movies
    total_clicks = sum([similar_user_movie_clicks[i][common_movies] for i in range(2)])

    # Sort the movies in descending order of total clicks
    recomendations = total_clicks.sort_values(ascending=False).index.tolist()

    # Add the top 5 movies to the list of recommendations
    for i in range(2):
        recomendations += similar_user_movie_clicks[i].index.tolist()
    
    # Remove duplicates from the list of recommendations
    recomendations = list(set(recomendations))
    
    #Remove movies already clicked on by the target user
    recomendations = [movie for movie in recomendations if movie not in user_movie_clicks[target_user].index.tolist()]

    # Cut the list to top 5 movies
    if len(recomendations) > 5:
        recomendations = recomendations[:5]

    return recomendations, top2_similar_users

In [79]:
data = 'archive/vodclickstream_uk_movies_03.csv'
df = pd.read_csv(data, sep=',', header=0, index_col=0)
df = df[df.genres != 'NOT AVAILABLE']

title                                            genres                                     
Wild Child                                       Comedy, Drama, Romance                         23
Zapped                                           Comedy, Family, Fantasy                        19
Barely Lethal                                    Action, Comedy                                 18
#Horror                                          Crime, Drama, Horror, Mystery, Thriller        16
Innocence                                        Fantasy, Horror, Mystery, Romance, Thriller    13
You Get Me                                       Crime, Drama, Romance, Thriller                12
Minor Details                                    Adventure, Family, Mystery                     12
Molly Moon and the Incredible Book of Hypnotism  Adventure, Family, Fantasy                     10
The 5th Wave                                     Action, Adventure, Sci-Fi, Thriller            10
IBOY            

Lets see how this works for a random user. We choose the `user_id` *'ffff2c5f9e'*.

In [84]:
target_user = 'ffff2c5f9e'

#Show the movies watched by the target user
df[df.user_id == target_user].groupby(['title', 'genres']).size().sort_values(ascending=False)[:10]

title              genres                           
Forks Over Knives  Documentary                          1
Hot Fuzz           Action, Comedy, Mystery, Thriller    1
dtype: int64

Now let's use the `Recommend` function to recommend at most five movies to the user.

In [100]:
recommended_movies, similar_users = Recomend(target_user, signatures, user_movie_clicks)
print(recommended_movies)

['The Woman with 7 Personalities', 'Natascha Kampusch: The Whole Story']


In [107]:
df[df.user_id == similar_users.index[0]].groupby(['title', 'genres']).size().sort_values(ascending=False)[:10]

title                               genres                           
Hot Fuzz                            Action, Comedy, Mystery, Thriller    2
Natascha Kampusch: The Whole Story  Documentary                          1
dtype: int64

In [108]:
df[df.user_id == similar_users.index[1]].groupby(['title', 'genres']).size().sort_values(ascending=False)[:10]

title                           genres                           
Hot Fuzz                        Action, Comedy, Mystery, Thriller    1
The Woman with 7 Personalities  Documentary                          1
dtype: int64

We see that the genres of the movies recommended are the same as the genres of the movies clicked by the user. Moreover, the movies recommended are the most clicked movies by the two most similar users and the movies already clicked on by the user are not recommended.

# 5. Algorithmic Question

### 5.a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on because he wants to do paid counseling for other desperate students: in a recursive fashion, the helped helps the helpable.

The first version of the code of the app is the following:

In [ ]:
from itertools import permutations
import numpy as np

# Function to find the highest score
def highest_score(S, marks):
    # Convert marks to numpy array
    marks = np.array(marks)
    
    # Iterate through the marks
    while len(marks):
        p = marks[0]
        # Check if the mark is greater or smaller than the current score
        if p > S:
            diff = p - S
            # Update the score
            S = p
            # Delete the mark from the array
            marks = np.delete(marks, np.where(marks == p))
            # Update the marks
            marks = marks - diff
        else:
            diff = S - p
            # Update the score
            S = p
            # Delete the mark from the array
            marks = np.delete(marks, np.where(marks == p))
            # Update the marks
            marks = marks + diff
    return S

S = 8
marks = [1, 5, 7]

# Initialize the max score and index
max_score = 0
ind = 0
# Find all the permutations
perms = list(permutations(marks, len(marks)))
# Iterate through the permutations
for num, i in enumerate(perms):
    # Find the highest score
    score = highest_score(S, i)
    if score > max_score:
        max_score = score
        ind = num
print(max_score, perms[ind])

### 5.b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university!

Federico is right, this code is very slow. Indeed, it is enough to consider that in order to compute all possible permutations, the algorithm has to perform $n!$ operations. This is a factorial, and it grows very fast. For example, if $n=10$, the algorithm has to perform $10! = 3,628,800$ operations. If $n=20$, the algorithm has to perform $20! = 2,432,902,008,176,640,000$ operations. Thus the complexity of the algorithm is $O(n!)$.

### 5.c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

We update the code and create a recursive function that doesn't consider all permutations, but handles them in a "smart" way. 

In [ ]:
import numpy as np
# Recursive function to find the highest score
def highest_score_recursive(S, marks):
    # Convert marks to numpy array
    marks = np.array(marks) # O(n)
    
    # Base case
    if len(marks) == 1: # O(1)
        return marks[0] # O(1)
    
    # Initialize the max score
    max_score = S # O(1)
    
    # Iterate through the marks
    for i in marks: # O(n)
        # Check if the mark is greater or smaller than the current score
        if i > S: # O(1)
            diff = i - S # O(1)
            # Update the marks
            marks1 = marks[marks != i] - diff # O(n)
            # Update the score
            S1 = i # O(1)
        else:
            diff = S - i # O(1)
            # Update the marks
            marks1 = marks[marks != i] + diff # O(n)
            # Update the score
            S1 = i # O(1)
        
        # Check if the length of the marks is even or odd
        if len(marks) % 2 == 0: # O(1)
            # Check if the marks are all smaller than the current score
            if all(marks1 < S1): #O(n)
                continue
            else:
                diff = max(marks1) - S1 # O(n)
                # Update the marks
                marks2 = marks1[marks1 != max(marks1)] - diff # O(n)
                # Update the score
                S2 = max(marks1) # O(n)
                # Find the highest score recursively
                max_score = max(max_score, highest_score_recursive(S2, marks2)) # * T(n-2)
        else:
            # Check if the marks are all greater than the current score
            if all(marks1 > S1): # O(n)
                continue
            else:
                diff = S1 - min(marks1) # O(n)
                # Update the marks
                marks2 = marks1[marks1 != min(marks1)] + diff # O(n)
                # Update the score
                S2 = min(marks1) # O(n)
                # Find the highest score recursively
                max_score = max(max_score, highest_score_recursive(S2, marks2)) # * T(n-2)
                
    return max_score

# Example usage with different S and marks lists
S1 = 8
marks1 = [1, 5, 7]
result1 = highest_score_recursive(S1, marks1)
print("The highest possible score for list 1 is:", result1)

S2 = 25
marks2 = [18, 24, 21, 32, 27]
result2 = highest_score_recursive(S2, marks2)
print("The highest possible score for list 2 is:", result2)

S3 = 30
marks3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result3 = highest_score_recursive(S3, marks3)
print("The highest possible score for list 3 is:", result3)

In [ ]:
%timeit highest_score_recursive(S3, marks3)

This code is already way faster than the first one. Let's compute the time complexity.

We denote the length of the `marks` list by $n$. 
- **Base case** (when $n=1$): in this case, the function returns `marks[0]`. This is a constant operation, so it is $O(1)$.
- **General case**: the function iterates through the `marks` list and performs certain operations for each element and then it calls itself recusrively for each element. In the worst case, for each element in the list, the function will create a new list of `marks` of length $n-2$ and call itself, generating a *recursive tree*. For both even and odd lists of `marks`, the height of the tree will be $\frac{n}{2}$. The number of nodes in the tree will be $n * (n-2) * (n-4) * ... * 1 = n!!$. Thus the number of recursive calls will be $n!!$. 
    - **Operations within each recursive call**: operations like array manipulations and finding minimum or maximum values within the list take $O(n)$ time in the worst case because they might need to traverse the entire list once.

The overall time complexity would be $O$(operations within each call × number of recursive calls).
Thus the overall time complexity of the algorithm is $O(n!!)$ as this is the term that dominates. Indeed,
$$
T(n) = T(n-2) * O(n) + O(n) \implies T(n) = T(n-4) * O(n-2) * O(n^2) + O(n) 
\\
\implies T(n) = T(n-6) * O(n-4) * O(n-2) * O(n) \implies ... \implies T(n) = T(1) * O(3) * O(5) * ... * O(n-2) * O(n) \implies T(n) = O(n!!)
$$


This is already a good improvement from before, indeed, if $n=10$, the algorithm has to perform $10!! = 3840$ operations, which is much less than $10! = 3,628,800$ operations. If $n=20$, the algorithm has to perform $20!! = 3715891200$ operations, which is much less than $20! = 2,432,902,008,176,640,000$ operations. However, this is still a very slow algorithm and we now we can do better, so before Federico gets even more angry, let's try to improve it even more.

One way to do so, could be using dynamic programming, in particular memoization. We can store the results of the recursive calls in a dictionary and use them to avoid recomputing the same results multiple times. This way, we can reduce the time complexity.

In [19]:
# Recursive function to find the highest score with memoization
def highest_score_recursive(S, marks, memo={}):
    
    # Convert marks to tuple to use as dictionary key
    marks = tuple(sorted(marks))
    
    # Check if result is already in memo
    if (S, marks) in memo:
        return memo[(S, marks)]
    
    # Base case
    if len(marks) == 1:
        return marks[0]
    
    # Initialize the max score
    max_score = S
    
    # Iterate through the marks
    for i in marks:
        # Check if the mark is greater or smaller than the current score
        if i > S:
            diff = i - S
            # Update the marks
            marks1 = [m for m in marks if m != i]
            marks1 = [m - diff for m in marks1]
            # Update the score
            S1 = i      
        else:
            diff = S - i
            # Update the marks
            marks1 = [m for m in marks if m != i]
            marks1 = [m + diff for m in marks1]
            # Update the score
            S1 = i
        
        # Check if the length of the marks is even or odd
        if len(marks) % 2 == 0:
            # Check if the marks are all smaller than the current score
            if all(m < S1 for m in marks1):
                continue
            else:
                diff = max(marks1) - S1
                # Update the marks
                marks2 = [m for m in marks1 if m != max(marks1)]
                marks2 = [m - diff for m in marks2]
                # Update the score
                S2 = max(marks1)
                # Find the highest score recursively
                max_score = max(max_score, highest_score_recursive(S2, marks2, memo))
        else:
            # Check if the marks are all greater than the current score
            if all(m > S1 for m in marks1):
                continue
            else:
                diff = S1 - min(marks1)
                # Update the marks
                marks2 = [m for m in marks1 if m != min(marks1)]
                marks2 = [m + diff for m in marks2]
                # Update the score
                S2 = min(marks1)
                # Find the highest score recursively
                max_score = max(max_score, highest_score_recursive(S2, marks2, memo))
    
    # Store result in memo before returning
    memo[(S, marks)] = max_score
    return max_score


# Example usage with different S and marks lists
S1 = 8
marks1 = [1, 5, 7]
result1 = highest_score_recursive(S1, marks1)
print("The highest possible score for list 1 is:", result1)


S2 = 25
marks2 = [18, 24, 21, 32, 27]
result2 = highest_score_recursive(S2, marks2)
print("The highest possible score for list 2 is:", result2)

S3 = 30
marks3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result3 = highest_score_recursive(S3, marks3)
print("The highest possible score for list 3 is:", result3)

The highest possible score for list 1 is: 11
The highest possible score for list 2 is: 44
The highest possible score for list 3 is: 205


In [20]:
%timeit highest_score_recursive(S3, marks3)

802 ns ± 77.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


This code is noticeably faster than the previous one. However, if the `marks` list is very long, the recursive tree will still be very deep and the algorithm will still be slow. Indeed, in the worst case, if not enough results are stored in the dictionary, the algorithm will still have to perform $n!!$ operations.
We need an approach that doesn't require a recursive tree if we want to handle very long lists of `marks`.

In [52]:
import numpy as np
def highest_score_opt(S, marks):
    
    pd = len(marks) # O(1)
    marks = np.array(marks) # O(n)
    
    index = 0 # O(1)
    while len(marks): # O(n)
        index += 1 # O(1)
        if (index % 2 == 0 and pd % 2 == 0) or (index % 2 != 0 and pd % 2 != 0): # O(1)
            p = max(marks) # O(n)
        else:
            p = min(marks) # O(n)
            
        if p > S: # O(1)
            marks = marks[marks != p] - (p - S) # O(n)
            S = p # O(1)
        else:
            marks = marks[marks != p] + (S - p) # O(n)
            S = p # O(1)
    return S

S1 = 8
marks1 = [1, 5, 7]
result1 = highest_score_opt(S1, marks1)
print("The highest possible score for list 1 is:", result1)


S2 = 25
marks2 = [18, 24, 21, 32, 27]
result2 = highest_score_opt(S2, marks2)
print("The highest possible score for list 2 is:", result2)

S3 = 30
marks3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result3 = highest_score_opt(S3, marks3)
print("The highest possible score for list 3 is:", result3)

The highest possible score for list 1 is: 11
The highest possible score for list 2 is: 44
The highest possible score for list 3 is: 205


In [53]:
%timeit highest_score(S3, marks3)

127 µs ± 9.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


The while loop is the most significant part of the code in terms of time complexity. It runs n times, where n is the length of the input list `marks`. Inside the loop, there are operations that also run in $O(n)$ time, such as `max(marks)`, `min(marks)`, and the array slicing and subtraction operations. 

Since these $O(n)$ operations are inside the loop that runs n times the overall time complexity is $O(n^2)$ for the while loop.

All other operations outside the loop run in constant time, $O(1)$.

So, the overall time complexity of the function is dominated by the while loop, which is $O(n^2)$.

### 5.d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!

In [99]:
import numpy as np
import heapq
from sortedcontainers import SortedSet

def highest_score_opt(S, marks):
    pd = len(marks)
    marks = np.array(marks)
    
    min_heap = list(marks)
    heapq.heapify(min_heap)
    
    max_heap = SortedSet(marks)
    
    index = 0
    while min_heap and max_heap:
        if (index % 2 == 0 and pd % 2 == 0) or (index % 2 != 0 and pd % 2 != 0):
            max_mark = max_heap.pop()
            min_heap = [mark - (max_mark - S) for mark in min_heap if mark != max_mark]
            max_heap = SortedSet(min_heap)
            S = max_mark
        else:
            min_mark = heapq.heappop(min_heap)
            min_heap = [mark + (S - min_mark) for mark in min_heap]
            max_heap = SortedSet(min_heap)
            S = min_mark
        index += 1
    return S

S1 = 8
marks1 = [1, 5, 7]
result1 = highest_score_opt(S1, marks1)
print("The highest possible score for list 1 is:", result1)


S2 = 25
marks2 = [18, 24, 21, 32, 27]
result2 = highest_score_opt(S2, marks2)
print("The highest possible score for list 2 is:", result2)

S3 = 30
marks3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result3 = highest_score_opt(S3, marks3)
print("The highest possible score for list 3 is:", result3)

The highest possible score for list 1 is: -1
The highest possible score for list 2 is: 4
The highest possible score for list 3 is: -145
